In [1]:
import numpy as np 
import pandas as pd 
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import csv
import plotly.graph_objects as go

# Open the CSV file using the csv module
with open('Environment_Temperature_change_E_All_Data_NOFLAG.csv', mode='r') as file:
    csv_reader = csv.reader(file)
    
    # Skip the header row
    header = next(csv_reader, None)
    
    # Load the data from csv_reader into a list
    data = [row for row in csv_reader]

# Convert the list to a Pandas DataFrame
df = pd.DataFrame(data, columns=header)

# Display the first few rows
print(df.head())


  Area Code         Area Months Code    Months Element Code  \
0         2  Afghanistan        7001   January         7271   
1         2  Afghanistan        7001   January         6078   
2         2  Afghanistan        7002  February         7271   
3         2  Afghanistan        7002  February         6078   
4         2  Afghanistan        7003     March         7271   

              Element Unit      Y1961     Y1962     Y1963  ...     Y2010  \
0  Temperature change   °C   0.777000  0.062000  2.744000  ...  3.601000   
1  Standard Deviation   °C   1.950000  1.950000  1.950000  ...  1.950000   
2  Temperature change   °C  -1.743000  2.465000  3.919000  ...  1.212000   
3  Standard Deviation   °C   2.597000  2.597000  2.597000  ...  2.597000   
4  Temperature change   °C   0.516000  1.336000  0.403000  ...  3.390000   

      Y2011      Y2012     Y2013      Y2014      Y2015     Y2016      Y2017  \
0  1.179000  -0.583000  1.233000   1.755000   1.943000  3.416000   1.201000   
1  1.9

In [2]:
def create_meteorological_year(df):
    # Extract the available years from the column names (e.g., 'Y1960')
    years = [col for col in df.columns if col.startswith('Y')]

    meteo_year_data = pd.DataFrame()

    for i in range(len(years) - 1):
        # Get current and next year (excluding 'Y')
        year = years[i][1:]  # Get the year as a string, removing 'Y'
        next_year = years[i + 1][1:]  # Next year for the November cutoff
        
        # Extract December of the previous year
        dec_prev_year = df[df['Months'] == 'December'][f'Y{year}']
        
        # Extract January to November of the current year
        jan_to_nov_current_year = df[df['Months'].isin(['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November'])][f'Y{next_year}']

        # Concatenate the December and the rest of the months
        meteo_year = pd.concat([dec_prev_year.reset_index(drop=True), jan_to_nov_current_year.reset_index(drop=True)], axis=0)

        # Add the reorganized year to a new DataFrame
        meteo_year_data[f'Meteorological Year {next_year}'] = meteo_year.reset_index(drop=True)

    return meteo_year_data

# Usage Example
df = pd.read_csv('Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding='ISO-8859-1')
meteo_data = create_meteorological_year(df)

# Display or save the output
print(meteo_data)


      Meteorological Year 1962  Meteorological Year 1963  \
0                        0.600                    -0.721   
1                        1.516                     1.516   
2                       -0.321                    -2.019   
3                        1.229                     1.229   
4                        0.906                    -0.303   
...                        ...                       ...   
6811                     0.322                     0.322   
6812                     0.765                     1.305   
6813                     0.635                     0.635   
6814                     0.668                     0.429   
6815                     0.435                     0.435   

      Meteorological Year 1964  Meteorological Year 1965  \
0                       -0.603                    -3.636   
1                        1.516                     1.516   
2                        0.884                     0.450   
3                        1.229         

In [3]:
def get_dec_jan_feb_data(df):
    # Extract the available years from the column names (e.g., 'Y1960')
    years = [col for col in df.columns if col.startswith('Y')]

    dec_jan_feb_data = pd.DataFrame()

    for i in range(len(years) - 1):
        # Get current and next year (excluding 'Y')
        year = years[i][1:]  # Get the year as a string, removing 'Y'
        next_year = years[i + 1][1:]  # Next year for Jan and Feb

        # Extract December of the current year
        dec_current_year = df[df['Months'] == 'December'][f'Y{year}']

        # Extract January and February of the next year
        jan_feb_next_year = df[df['Months'].isin(['January', 'February'])][f'Y{next_year}']

        # Concatenate December and January-February data
        dec_jan_feb = pd.concat([dec_current_year.reset_index(drop=True), jan_feb_next_year.reset_index(drop=True)], axis=0)

        # Add the data to the result DataFrame, labeled by the next year (for Jan-Feb context)
        dec_jan_feb_data[f'Dec-Jan-Feb {next_year}'] = dec_jan_feb.reset_index(drop=True)

    return dec_jan_feb_data

# Usage Example
df = pd.read_csv('Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding='ISO-8859-1')
dec_jan_feb_data = get_dec_jan_feb_data(df)

# Display or save the output
print(dec_jan_feb_data)


      Dec-Jan-Feb 1962  Dec-Jan-Feb 1963  Dec-Jan-Feb 1964  Dec-Jan-Feb 1965  \
0                0.600            -0.721            -0.603            -3.636   
1                1.516             1.516             1.516             1.516   
2               -0.321            -2.019             0.884             0.450   
3                1.229             1.229             1.229             1.229   
4                0.906            -0.303             1.938            -0.706   
...                ...               ...               ...               ...   
1699             0.493             0.493             0.493             0.493   
1700            -0.168            -0.906             0.532             0.176   
1701             0.583             0.583             0.583             0.583   
1702            -0.342            -0.018             0.125            -1.068   
1703             0.930             0.930             0.930             0.930   

      Dec-Jan-Feb 1966  Dec-Jan-Feb 196

In [4]:
def get_mar_apr_may_data(df):
    # Extract the available years from the column names (e.g., 'Y1960')
    years = [col for col in df.columns if col.startswith('Y')]

    mar_apr_may_data = pd.DataFrame()

    for year in years:
        # Extract March, April, and May of the current year
        mar_apr_may = df[df['Months'].isin(['March', 'April', 'May'])][year]

        # Add the data to the result DataFrame, labeled by the year
        mar_apr_may_data[f'Mar-Apr-May {year[1:]}'] = mar_apr_may.reset_index(drop=True)

    return mar_apr_may_data

# Usage Example
df = pd.read_csv('Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding='ISO-8859-1')
mar_apr_may_data = get_mar_apr_may_data(df)

# Display or save the output
print(mar_apr_may_data)


      Mar-Apr-May 1961  Mar-Apr-May 1962  Mar-Apr-May 1963  Mar-Apr-May 1964  \
0                0.516             1.336             0.403             1.659   
1                1.512             1.512             1.512             1.512   
2               -1.709             0.117             0.919            -0.533   
3                1.406             1.406             1.406             1.406   
4                1.412            -0.092            -0.690            -0.160   
...                ...               ...               ...               ...   
1699             0.677             0.677             0.677             0.677   
1700            -0.029            -0.054             0.423            -0.307   
1701             0.411             0.411             0.411             0.411   
1702            -0.290            -0.320             0.135             0.128   
1703             0.383             0.383             0.383             0.383   

      Mar-Apr-May 1965  Mar-Apr-May 196

In [5]:
def get_jun_jul_aug_data(df):
    # Extract the available years from the column names (e.g., 'Y1960')
    years = [col for col in df.columns if col.startswith('Y')]

    jun_jul_aug_data = pd.DataFrame()

    for year in years:
        # Extract June, July, and August of the current year
        jun_jul_aug = df[df['Months'].isin(['June', 'July', 'August'])][year]

        # Add the data to the result DataFrame, labeled by the year
        jun_jul_aug_data[f'Jun-Jul-Aug {year[1:]}'] = jun_jul_aug.reset_index(drop=True)

    return jun_jul_aug_data

# Usage Example
df = pd.read_csv('Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding='ISO-8859-1')
jun_jul_aug_data = get_jun_jul_aug_data(df)

# Display or save the output
print(jun_jul_aug_data)


      Jun-Jul-Aug 1961  Jun-Jul-Aug 1962  Jun-Jul-Aug 1963  Jun-Jul-Aug 1964  \
0               -0.058            -1.061             1.164            -0.519   
1                0.930             0.930             0.930             0.930   
2                0.884             0.292             0.348            -0.142   
3                0.585             0.585             0.585             0.585   
4                0.391            -0.220             0.094             0.671   
...                ...               ...               ...               ...   
1699             0.276             0.276             0.276             0.276   
1700            -0.112            -0.181             0.012             0.239   
1701             0.343             0.343             0.343             0.343   
1702             0.064             0.130             0.036            -0.361   
1703             0.322             0.322             0.322             0.322   

      Jun-Jul-Aug 1965  Jun-Jul-Aug 196

In [6]:
def get_sep_oct_nov_data(df):
    # Extract the available years from the column names (e.g., 'Y1960')
    years = [col for col in df.columns if col.startswith('Y')]

    sep_oct_nov_data = pd.DataFrame()

    for year in years:
        # Extract September, October, and November of the current year
        sep_oct_nov = df[df['Months'].isin(['September', 'October', 'November'])][year]

        # Add the data to the result DataFrame, labeled by the year
        sep_oct_nov_data[f'Sep-Oct-Nov {year[1:]}'] = sep_oct_nov.reset_index(drop=True)

    return sep_oct_nov_data

# Usage Example
df = pd.read_csv('Environment_Temperature_change_E_All_Data_NOFLAG.csv', encoding='ISO-8859-1')
sep_oct_nov_data = get_sep_oct_nov_data(df)

# Display or save the output
print(sep_oct_nov_data)


      Sep-Oct-Nov 1961  Sep-Oct-Nov 1962  Sep-Oct-Nov 1963  Sep-Oct-Nov 1964  \
0                1.445            -1.797             0.163            -0.253   
1                0.832             0.832             0.832             0.832   
2               -1.102            -0.968             1.654            -2.839   
3                1.310             1.310             1.310             1.310   
4               -0.540            -2.171             0.584            -0.574   
...                ...               ...               ...               ...   
1699             0.322             0.322             0.322             0.322   
1700             0.090             0.765             1.305            -0.355   
1701             0.635             0.635             0.635             0.635   
1702            -0.023             0.668             0.429            -0.003   
1703             0.435             0.435             0.435             0.435   

      Sep-Oct-Nov 1965  Sep-Oct-Nov 196

In [7]:
csv_file_path = 'sep_oct_nov_data.csv'
sep_oct_nov_data.to_csv(csv_file_path, index=False)

csv_file_path

'sep_oct_nov_data.csv'

In [8]:
csv_file_path = 'jun_jul_aug_data.csv'
jun_jul_aug_data.to_csv(csv_file_path, index=False)

csv_file_path

'jun_jul_aug_data.csv'

In [9]:
csv_file_path = 'jun_jul_aug_data.csv'
sep_oct_nov_data.to_csv(csv_file_path, index=False)

csv_file_path

'jun_jul_aug_data.csv'

In [10]:
csv_file_path = 'dec_jan_feb_data.csv'
dec_jan_feb_data.to_csv(csv_file_path, index=False)

csv_file_path

'dec_jan_feb_data.csv'

In [11]:
csv_file_path = 'meteo_data.csv'
meteo_data.to_csv(csv_file_path, index=False)

csv_file_path

'meteo_data.csv'

In [12]:
# Let's first load the CSV file to see its contents and understand the data structure.
import csv

file_path = 'meteo_data.csv'

# Reading the first few lines of the CSV file to inspect its structure
with open(file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Extracting the header
    first_row = next(reader)  # Extracting the first row of data

header, first_row


(['Meteorological Year 1962',
  'Meteorological Year 1963',
  'Meteorological Year 1964',
  'Meteorological Year 1965',
  'Meteorological Year 1966',
  'Meteorological Year 1967',
  'Meteorological Year 1968',
  'Meteorological Year 1969',
  'Meteorological Year 1970',
  'Meteorological Year 1971',
  'Meteorological Year 1972',
  'Meteorological Year 1973',
  'Meteorological Year 1974',
  'Meteorological Year 1975',
  'Meteorological Year 1976',
  'Meteorological Year 1977',
  'Meteorological Year 1978',
  'Meteorological Year 1979',
  'Meteorological Year 1980',
  'Meteorological Year 1981',
  'Meteorological Year 1982',
  'Meteorological Year 1983',
  'Meteorological Year 1984',
  'Meteorological Year 1985',
  'Meteorological Year 1986',
  'Meteorological Year 1987',
  'Meteorological Year 1988',
  'Meteorological Year 1989',
  'Meteorological Year 1990',
  'Meteorological Year 1991',
  'Meteorological Year 1992',
  'Meteorological Year 1993',
  'Meteorological Year 1994',
  'Meteoro

In [13]:
import plotly.graph_objects as go
import numpy as np

# Creating simulated datasets for demonstration
np.random.seed(0)  # For reproducibility
years = [int(year.split()[-1]) for year in header]
values = [float(value) for value in first_row]

# Simulating two additional datasets by adding some random noise
annex_i_values = np.array(values) + np.random.normal(0, 0.5, len(values))
non_annex_i_values = np.array(values) + np.random.normal(0, 0.7, len(values))

# Create the area chart with Plotly
fig = go.Figure()

# Use area traces
fig.add_trace(go.Scatter(x=years, y=values,
                         mode='lines+markers+text',
                         name='World',
                         fill='tozeroy',  # Fill the area under the line
                         marker_color='blue',
                         line=dict(width=0.5)))  # Adjust line width

fig.add_trace(go.Scatter(x=years, y=annex_i_values,
                         mode='lines+markers+text',
                         name='Annex I countries',
                         fill='tozeroy',  # Fill the area under the line
                         marker_color='orange',
                         line=dict(width=0.5)))  # Adjust line width

fig.add_trace(go.Scatter(x=years, y=non_annex_i_values,
                         mode='lines+markers+text',
                         name='Non-Annex I countries',
                         fill='tozeroy',  # Fill the area under the line
                         marker_color='green',
                         line=dict(width=0.5)))  # Adjust line width

# Update layout for area chart
fig.update_layout(title='Meteorological Data Over Years (Area Chart)',
                  xaxis_title='Years',
                  yaxis_title='Meteorological Value',
                  showlegend=True,
                  legend_title='Country Classification',
                  plot_bgcolor='lightgrey',
                  paper_bgcolor='white')

# Show the figure
fig.show()

